In [7]:
import json
from transformers import AutoModelForSeq2SeqLM, PreTrainedTokenizerFast, AutoTokenizer
from transformers.models.bart.modeling_bart import BartForConditionalGeneration

# load data
file_path = "task1/train/eLife_train.jsonl"
with open(file_path, 'r') as f:
    lst = []
    for line in f.readlines():
        text = json.loads(line)
        lst.append(text)

In [19]:
# load model
tokenizer = PreTrainedTokenizerFast.from_pretrained("facebook/bart-base") # no <pad> token
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
bart = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

'<s>'

In [45]:
# text encoding
# with tokenizer.as_target_tokenizer(): # same behaviour with/w.o context manager
summary = tokenizer([lst[0]['lay_summary']], return_tensors="pt", padding="longest" ) 
target_ids, target_mask = summary["input_ids"], summary["attention_mask"] # (bsz, target_seq_len)

article = tokenizer([lst[0]['article']], return_tensors="pt", padding="max_length", truncation=True )
input_ids, attention_mask = article["input_ids"], article["attention_mask"] # (bsz, 1024)

In [51]:
import torch.functional as F
def sequence_cross_entropy_with_logits(logits, shifted_target_ids, shifted_target_mask):
    # flatten
    logits_flat = logits.view(-1, logits.size(-1))
    targets_flatten = shifted_target_ids.view(-1)
    return F.cross_entropy(logits_flat, targets_flatten, shifted_target_mask)


torch.Size([1, 371])

In [60]:
from typing import cast
import torch
bart_output = bart(
    input_ids=input_ids, 
    attention_mask=attention_mask, 
    decoder_input_ids=target_ids[:, :-1].contiguous(),
    decoder_attention_mask=target_mask[:, :-1].contiguous(),
    use_cache=False,
    return_dict=True 
)
logits = bart_output.logits # (bsz, target_seq_len-1, vocab_size), '-1' for the last position
shifted_target_ids =  target_ids[:, 1:].type(torch.LongTensor).contiguous() # (bsz, target_seq_len-1, vocab_size), '-1' for the first position
shifted_target_mask = target_mask[:, 1:].type(torch.LongTensor).contiguous()
loss = sequence_cross_entropy_with_logits(logits, shifted_target_ids, shifted_target_mask, shifted_target_mask)


torch.Size([1, 372])
torch.Size([1, 371, 50265])
